In [ ]:
# Package imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [ ]:
def read_data():
    df = pd.DataFrame()
    dirs = os.listdir('data')
    for folder in dirs:
        if folder != '.DS_Store':
            filenames = glob.glob(os.path.join('data', folder, '*/*.txt'), recursive = True)
            for filename in filenames:
                if ('MANIFEST.txt' not in filename) and ('annotations.txt' not in filename):
                    temp_df = pd.read_csv(filename, '\t')
                    row_dict = {'cancer': folder}
                    for index, row in temp_df.iterrows():
                        row_dict[row['miRNA_ID']] = row['reads_per_million_miRNA_mapped']

                    df = df.append(row_dict, ignore_index = True)
    
    le = preprocessing.LabelEncoder()
    df['cancer_code'] = le.fit_transform(df['cancer'])
    
    df.to_csv('data_cleaned.csv', index=False)


In [ ]:
def load_data():
    return pd.read_csv('data_cleaned.csv')

In [7]:
def random_forest():
    pass

In [ ]:
def svm():
    pass

In [8]:
def main():
    df = load_data()
    
    #split into train, test, and evaluation sets
    X = df.drop(columns=['cancer', 'cancer_code'])
    y = df['cancer_code']
    X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=0.2, random_state=2, stratify=y)
    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.25, random_state=2, stratify=y_train)
    
    #scale data
    min_max_scalar = preprocessing.MinMaxScaler()
    X_train_scaled = min_max_scalar.fit_transform(X_train)
    min_max_scalar = preprocessing.MinMaxScaler()
    X_test_scaled = min_max_scalar.fit_transform(X_test)
    min_max_scalar = preprocessing.MinMaxScaler()
    X_eval_scaled = min_max_scalar.fit_transform(X_eval)

In [ ]:
main()